In [4]:
# [IF NEEDED], install prerequisites
import sys
!{sys.executable} -m pip install pip install neuroconv[intan]
!{sys.executable} -m pip install pip install pynwb

Defaulting to user installation because normal site-packages is not writeable


In [145]:
# CREATED: 28-NOV-2022
# LAST EDIT: 29-NOV-2022
# AUTHOR: DUANE RINEHART, MBA (drinehart@ucsd.edu)

# PRE-PROCESS ELECTRODE MAPPINGS (Excel or .cfg -> Standard format Excel with 2 columns: device_channel	intan_channel)

import os, math, time, pynwb, sys, re
from pathlib import Path, PurePath, PureWindowsPath
from datetime import datetime, timedelta
from dateutil.tz import tzlocal
import pandas as pd
from pathlib import PurePath

from ConvertIntanToNWB import convert_to_nwb

#################################################################
# APP CONSTANTS
excel_file = 'input.xlsx'
base_path = Path('/mnt/e/temp/Devor-gdrive/')
infile = Path(base_path, excel_file)
output_path = Path('/mnt/e/temp/Devor-gdrive/output/')
debug = True
#################################################################

def load_data():
    '''Used for meta-data loading'''
    lstNWBFields = ['session_id', 'age', 'subject_description', 'species', 'genotype', 'sex', 'subject_strain', 'subject_weight', 'session_description', 'subject_id', 'pharmacology', 'date_of_birth(YYYY-MM-DD)', 'src_folder_directory', 'stimulus_notes_include', 'stimulus_notes_paradigm', 'stimulus_notes_direct_electrical_stimulation', 'stimulus_notes_direct_electrical_stimulation_paradigm', 'pharmacology_notes_anesthetized_during_recording', 'electrode_recordings', 'electrode_recordings_type', 'electrode_recordings_contact_material', 'electrode_recordings_substrate'] #headers I need

    lstExtractionFields = pd.read_excel(infile, sheet_name="auto", usecols=lstNWBFields) #just extract columns/fields I need
    return lstExtractionFields


def get_measurements_data(src_folder_directory: str, channel_map_recordings_file: str, session_id: str, debug: bool = False):
    '''Used for electrode measurements table processing'''
    electrode_data = None

    if Path(channel_map_recordings_file).suffix == '.xls' or Path(channel_map_recordings_file).suffix == '.xlsx':
        #Note: must use 'PureWindowsPath' for org dataset (base_directory)
        #N.B. must use Path to extract data
        base_directory = PureWindowsPath(src_folder_directory).parts[:-1] #remove last part of path
        input_filename = Path(base_path, *base_directory, channel_map_recordings_file) #must use POSIX path (WSL)
        if debug == True:
            print(f"READING ELECTRODE MEASUREMENT DATA FROM FILE: {input_filename}")

        if 'topview' in channel_map_recordings_file:
            #'topview' files have defined electrode mappings in 'Device Channel' and 'Intan Channel' columns
            lstNWBFields = ['Device Channel', 'Intan Channel'] #headers I need
            mapping_values = pd.read_excel(input_filename, usecols=lambda x: x in lstNWBFields) #just extract columns/fields I need (ignore if not present - lambda part)
            mapping_indexed_tuples = list(zip(mapping_values['Device Channel'], mapping_values['Intan Channel']))
        else:
            lstNWBFields = ['date', 'runnum', 'array', 'set', 'mapping', 'ep', 'epFile', 'stim', 'stimType', 'stimCondition', 'stimLocation', 'stimFile', 'imaging', 'imagingFile', 'imagingType', 'imagingCond', 'imagingLocation', 'webcam', 'webcamFile', 'trialAvg', 'trialAvgType', 'trialAvgCond', 'animal', 'comment'] #headers I need
            electrode_data = pd.read_excel(input_filename, usecols=lambda x: x in lstNWBFields) #just extract columns/fields I need (ignore if not present - lambda part)
            filter_results = electrode_data[electrode_data['epFile'] == session_id] #filter dataframe for relevant measurement data (where session_id matches epFile)

            str_mapping_values = filter_results.iloc[0]['mapping'] #Intan channels; ordinal position is 'device channel'
            mapping_values = str_mapping_values.strip('][').split(',') #read as string from Excel
            mapping_values = ['A' + individual_val for individual_val in mapping_values] #preface each value w/ 'A' with list comprehension
            mapping_indexed_tuples = list(enumerate(mapping_values)) #create indexed tuple

    return mapping_indexed_tuples


#start
lstRecords = load_data().to_dict('records') #creates list of dictionaries

for cnt, dataset in enumerate(lstRecords):
    print(f"PROCESSING ELECTRODE MAPPINGS #{cnt+1}")

    channel_map_recordings_file = dataset['electrode_recordings']
    src_path_supplement = PureWindowsPath(dataset['src_folder_directory'])
    print(f"ELECTRODE MAPPINGS SOURCE FILE: {channel_map_recordings_file}")

    session_id = dataset['session_id'] #only use relevant recording
    mapping_indexed_tuples = get_measurements_data(src_path_supplement, channel_map_recordings_file, session_id, debug)

    src_folder_directory = dataset['src_folder_directory']
    output_folder = PureWindowsPath(src_folder_directory).parts[:-1]
    output_filename = 'electrode_mappings.xlsx'
    Path(output_path, *output_folder).mkdir(parents=True, exist_ok=True) # CREATE FOLDER STRUCTURE IF NOT EXISTS
    dest_path = str(Path(output_path, *output_folder, output_filename)) # CREATE EXCEL STANDARDIZED MAPPINGS FILE, IF NOT EXISTS

    if os.path.isfile(dest_path) != True:
        df = pd.DataFrame(mapping_indexed_tuples, columns =['device_channel', 'intan_channel'])
        df.to_excel(dest_path, index=False)


PROCESSING ELECTRODE MAPPINGS #1
ELECTRODE MAPPINGS SOURCE FILE: 20191107_San4_day21_EXP.xls
READING ELECTRODE MEASUREMENT DATA FROM FILE: /mnt/e/temp/Devor-gdrive/20191017_Chronic implant_SL2701_San4/20191107_San4_day21/20191107_San4_day21_EXP.xls
return,('20191017_Chronic implant_SL2701_San4', '20191107_San4_day21')
PROCESSING ELECTRODE MAPPINGS #2
ELECTRODE MAPPINGS SOURCE FILE: 20191113_San4_day27_EXP.xls
READING ELECTRODE MEASUREMENT DATA FROM FILE: /mnt/e/temp/Devor-gdrive/20191017_Chronic implant_SL2701_San4/20191113_San4_day27/20191113_San4_day27_EXP.xls
return,('20191017_Chronic implant_SL2701_San4', '20191113_San4_day27')
PROCESSING ELECTRODE MAPPINGS #3
ELECTRODE MAPPINGS SOURCE FILE: 20191127_San4_day41_EXP.xls
READING ELECTRODE MEASUREMENT DATA FROM FILE: /mnt/e/temp/Devor-gdrive/20191017_Chronic implant_SL2701_San4/20191127_San4_day41/20191127_San4_day41_EXP.xls
return,('20191017_Chronic implant_SL2701_San4', '20191127_San4_day41')
PROCESSING ELECTRODE MAPPINGS #4
ELECTRO

In [146]:
# CREATED: 23-NOV-2022
# LAST EDIT: 23-NOV-2022
# AUTHOR: DUANE RINEHART, MBA (drinehart@ucsd.edu)

# MANUAL EXAMINE CONTENTS OF NWB FILE

from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile
from pynwb import TimeSeries
from pynwb import NWBHDF5IO
import numpy as np
from hdmf.backends.hdf5.h5_utils import H5DataIO
from pynwb import NWBHDF5IO


#ref: https://github.com/NeurodataWithoutBorders/pynwb/issues/1297

#NOTE: filename REQUIRES COMPLETE PATH

filename = 'ecephys_tutorial.nwb'
with NWBHDF5IO(filename, "r") as io:
    f3 = io.read()
    print(f"NWB FILE CONTENTS {filename}")
    print(f3)

NWB FILE CONTENTS ecephys_tutorial.nwb
root pynwb.file.NWBFile at 0x140267304700416
Fields:
  devices: {
    SL2701 <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    electrode_mapping <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  file_create_date: [datetime.datetime(2022, 11, 29, 23, 36, 1, 629388, tzinfo=tzutc())]
  identifier: NWBE4
  session_description: demonstrate external files
  session_start_time: 2022-11-29 23:36:01.629384+00:00
  timestamps_reference_time: 2022-11-29 23:36:01.629384+00:00

